In [66]:
# -*- coding: utf-8 -*-
import sys
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [67]:
print("Entradas do Treinamento: {}, labels: {}".format(len(train_data), len(train_labels)))

Entradas do Treinamento: 25000, labels: 25000


In [89]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
word_index = dict([(key, value) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

def encode_review(text):
    sentences = clean_text(text)
    word_index = imdb.get_word_index()
    x_test = [[self.word_index[w] for w in sentences if w in self.word_index]]
    x_test = pad_sequences(x_test, maxlen=256) # Should be same which you used for training data
    vector = np.array([x_test.flatten()])
    
    return vector

In [69]:
decode_review(train_data[0])


u"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for w

In [70]:
predict_data = test_data
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [71]:
len(train_data[0]), len(train_data[1])


(256, 256)

In [72]:
print(train_data[0])


[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [73]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 2s 154us/step - loss: 0.6919 - acc: 0.5143 - val_loss: 0.6902 - val_acc: 0.5573
Epoch 2/40
15000/15000 [==============================] - 1s 44us/step - loss: 0.6862 - acc: 0.6481 - val_loss: 0.6820 - val_acc: 0.7332
Epoch 3/40
15000/15000 [==============================] - 1s 41us/step - loss: 0.6731 - acc: 0.7520 - val_loss: 0.6655 - val_acc: 0.7521
Epoch 4/40
15000/15000 [==============================] - 1s 47us/step - loss: 0.6496 - acc: 0.7747 - val_loss: 0.6390 - val_acc: 0.7665
Epoch 5/40
15000/15000 [==============================] - 1s 45us/step - loss: 0.6144 - acc: 0.8003 - val_loss: 0.6012 - val_acc: 0.7895
Epoch 6/40
15000/15000 [==============================] - 1s 42us/step - loss: 0.5696 - acc: 0.8179 - val_loss: 0.5578 - val_acc: 0.8063
Epoch 7/40
15000/15000 [==============================] - 1s 42us/step - loss: 0.5186 - acc: 0.8355 - val_loss: 0.5096 - val_ac

In [75]:
results = model.evaluate(test_data, test_labels)

print(results)

25000/25000 [==============================] - 1s 21us/step
[0.3396013977622986, 0.87228]


In [92]:
# 50 é pra ser negativo

predict_test = predict_data[50]

print(decode_review(predict_test))

print(model.predict(np.array([predict_test])))

predict_test = predict_data[16]

print(decode_review(predict_test))

print(model.predict(np.array([predict_test])))


<START> first of i should point out that i used to love <UNK> the <UNK> as a child and i really enjoyed the movie even though i am in my <UNK> br br but this movie was so bad i was ashamed to have been a fan in my youth br br ok ok i know this is a movie for kids and isn't aimed at people like me anyway but this is my thoughts on the movie for other people of my age br br the main downfall in this film is the itself it has to be the most annoying character i have seen in a child's movie possibly even more annoying then the young child in monsters inc it has the most annoying voice and <UNK> around singing stupid things and making even more stupid comments i know <UNK> movies aren't exactly high brow but this was insulting to even a 2 year <UNK> intelligence br br secondly where was the story previous <UNK> <UNK> had a least a point to the story yes i can see this was about accepting people who are different to you into your hearts but really it ended and i felt like i had watched a 5 m

NameError: global name 'clean_text' is not defined